## Vectorized Backtester

In [1]:
import pandas as pd
import vectorbt as vbt
from IPython.display import Markdown, display

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
start="2015-12-31 UTC"
end="2024-12-31 UTC"
prices = vbt.YFData.download(["AAPL", "AMZN", "META", "NFLX", "GOOG"], start=start, end=end).get("Close")

In [4]:
fast_ma = vbt.MA.run(prices, 10, short_name="short")
slow_ma = vbt.MA.run(prices, 30, short_name="long")

In [5]:
entries = fast_ma.ma_crossed_above(slow_ma)
display(entries)

short_window                  10                            
long_window                   30                            
symbol                      AAPL   AMZN   META   NFLX   GOOG
Date                                                        
2015-12-31 05:00:00+00:00  False  False  False  False  False
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2024-12-23 05:00:00+00:00  False  False  False  False  False
2024-12-24 05:00:00+00:00  False  False  False  False  False
2024-12-26 05:00:00+00:00  False  False  False  False  False
2024-12-27 05:00:00+00:00  False  False  False  False  False
2024-12-30 05:00:00+00:00  False  False  False  False  False

[2264 rows x 5 columns]

In [6]:
exits = fast_ma.ma_crossed_below(slow_ma)
display(exits)

short_window                  10                            
long_window                   30                            
symbol                      AAPL   AMZN   META   NFLX   GOOG
Date                                                        
2015-12-31 05:00:00+00:00  False  False  False  False  False
2016-01-04 05:00:00+00:00  False  False  False  False  False
2016-01-05 05:00:00+00:00  False  False  False  False  False
2016-01-06 05:00:00+00:00  False  False  False  False  False
2016-01-07 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2024-12-23 05:00:00+00:00  False  False  False  False  False
2024-12-24 05:00:00+00:00  False  False  False  False  False
2024-12-26 05:00:00+00:00  False  False  False  False  False
2024-12-27 05:00:00+00:00  False  False  False  False  False
2024-12-30 05:00:00+00:00  False  False  False  False  False

[2264 rows x 5 columns]

In [7]:
port = vbt.Portfolio.from_signals(prices, entries, exits)

In [8]:
display(port.stats())

Start                         2015-12-31 05:00:00+00:00
End                           2024-12-30 05:00:00+00:00
Period                                             2264
Start Value                                       100.0
End Value                                    372.463488
Total Return [%]                             272.463488
Benchmark Return [%]                         615.333763
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              43.728173
Max Drawdown Duration                             708.2
Total Trades                                       39.4
Total Closed Trades                                38.4
Total Open Trades                                   1.0
Open Trade PnL                                27.651513
Win Rate [%]                                  48.201368
Best Trade [%]                                66.334753
Worst Trade [%]                              -13

In [9]:
port.total_return().groupby("symbol").mean().vbt.barplot()

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'f616caa3-8ea9-478f-8561-3c7726a7f473',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([4.72951995, 3.85038548, 0.64198625, 3.29751764, 1.10376506])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [10]:
vbt.Portfolio.from_holding(prices, freq="1d").total_return().groupby("symbol").mean().vbt.barplot()

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '79eb717d-437a-4cd0-b1e6-0ed801bd25fc',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([9.5787683 , 5.54840324, 4.09652009, 4.67072854, 6.872268  ])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [11]:
mult_prices, _ = prices.vbt.range_split(n=4)

In [12]:
fast_ma = vbt.MA.run(mult_prices, [10, 20], short_name="fast")
slow_ma = vbt.MA.run(mult_prices, [30, 30], short_name="slow")

In [13]:
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

In [14]:
port = vbt.Portfolio.from_signals(mult_prices, entries, exits, freq="1D")

In [15]:
port.total_return().groupby(["split_idx", "symbol"]).mean().unstack(level=-1).vbt.barplot()

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'bar',
              'uid': '0679f10b-8e7a-46bd-9413-f1cab6f9156f',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([0.30232389, 1.4149583 , 0.10669801, 0.28168666])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'bar',
              'uid': 'aed105ab-d228-4617-85d2-7f326d3cbbb6',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 1.03913867,  0.43469221, -0.24683386,  0.90073535])},
             {'name': 'GOOG',
              'showlegend': True,
              'type': 'bar',
              'uid': '88ff0367-dc4f-4af4-a89d-3e851ff3e957',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([0.0398932 , 0.32271836, 0.01128335, 0.20246764])},
             {'name': 'META',
              'showlegend': True,
              'type': 'bar',
              'uid': '4a66c5f9-940c-

In [16]:
display(port.orders.stats(group_by=True))

Start                                0
End                                565
Period               566 days 00:00:00
Total Records                      738
Total Buy Orders                   381
Total Sell Orders                  357
Min Size                      0.129586
Max Size                      4.706515
Avg Size                      1.130154
Avg Buy Size                  1.124771
Avg Sell Size                   1.1359
Avg Buy Price               178.938912
Avg Sell Price              176.683989
Total Fees                         0.0
Min Fees                           0.0
Max Fees                           0.0
Avg Fees                           0.0
Avg Buy Fees                       0.0
Avg Sell Fees                      0.0
Name: group, dtype: object

In [17]:
display(port.sharpe_ratio())

fast_window  slow_window  split_idx  symbol
10           30           0          AAPL      1.385101
                                     AMZN      1.734586
                                     META      0.111586
                                     NFLX      0.527159
                                     GOOG      0.559279
                          1          AAPL      2.689861
                                     AMZN      1.112981
                                     META      1.034130
                                     NFLX      0.293467
                                     GOOG      0.922784
                          2          AAPL      0.315057
                                     AMZN     -0.680440
                                     META     -0.451229
                                     NFLX     -0.611166
                                     GOOG      0.063534
                          3          AAPL      0.954340
                                     AMZN      1.404871
    

## Forward optimization

In [18]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt

In [19]:
start = "2015-12-31 UTC"
end = "2019-12-31 UTC"
prices = vbt.YFData.download("AAPL", start=start, end=end).get("Close")

In [20]:
(in_price, in_indexes), (out_price, out_indexes) = prices.vbt.rolling_split(n=30,
                                                                            window_len=365 * 2,
                                                                            set_lens=(180,),
                                                                            left_to_right=False)

In [21]:
def sim_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(price,
                                        windows,
                                        r=2,
                                        short_names=["fast", "slow"])
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    port = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return port.sharpe_ratio()

In [22]:
def get_best_index(performance):
    return performance[performance.groupby("split_idx").idxmax()].index

def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [23]:
def sim_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    fast_ma = vbt.MA.run(price, window=best_fast_windows, per_column=True)
    slow_ma = vbt.MA.run(price, window=best_slow_windows, per_column=True)
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    port = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return port.sharpe_ratio()

### In-sample

In [24]:
windows = np.arange(10, 40)
in_sharpe = sim_all_params(in_price, windows, direction="both", freq="d")

In [25]:
in_best_index = get_best_index(in_sharpe)
in_best_fast_windows = get_best_params(in_best_index, "fast_window")
in_best_slow_windows = get_best_params(in_best_index, "slow_window")
in_best_window_pairs = np.array(list(zip(in_best_fast_windows, in_best_slow_windows)))

### Out-of-sample

In [26]:
out_test_sharpe = sim_best_params(out_price,
                                  in_best_fast_windows,
                                  in_best_slow_windows,
                                  direction="both",
                                  freq="d")
display(out_test_sharpe)

ma_window  ma_window  split_idx
10         11         0           -0.020974
12         13         1            0.322699
                      2            0.751386
10         11         3            1.477455
12         13         4            1.161412
10         11         5            2.040427
                      6            2.026685
18         23         7            2.732003
                      8            2.147848
                      9            2.236980
                      10           2.546842
                      11           2.714035
                      12           2.487526
                      13           2.815397
                      14           2.579146
                      15           1.150049
24         25         16           0.358654
23         26         17           0.722216
                      18           1.001097
24         25         19           0.312880
                      20          -0.004982
                      21          -0.531237


In [27]:
dir(port)

['__annotations__',
 '__cached_asset_flow',
 '__cached_asset_value',
 '__cached_assets',
 '__cached_benchmark_returns',
 '__cached_benchmark_value',
 '__cached_cash',
 '__cached_cash_flow',
 '__cached_get_filled_close',
 '__cached_get_init_cash',
 '__cached_get_orders',
 '__cached_get_returns_acc',
 '__cached_orders',
 '__cached_returns',
 '__cached_sharpe_ratio',
 '__cached_total_profit',
 '__cached_total_return',
 '__cached_value',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_seq',
 '_cash_sharing',
 '_close',
 '_config',
 '_fillna_close',
 '_iloc',
 '_indexing_kwargs',
 '_init_cash',
 '_loc',
 '_log_records',
 '_metrics',
 '_order_records',

In [28]:
in_sample_best = in_sharpe[in_best_index].values
out_sample_test = out_test_sharpe.values
t, p = stats.ttest_ind(a=out_sample_test,
                       b=in_sample_best,
                       alternative="greater")
display(t, p)

-1.0383533180444202

0.8482924329427151

In [29]:
display(out_test_sharpe)

ma_window  ma_window  split_idx
10         11         0           -0.020974
12         13         1            0.322699
                      2            0.751386
10         11         3            1.477455
12         13         4            1.161412
10         11         5            2.040427
                      6            2.026685
18         23         7            2.732003
                      8            2.147848
                      9            2.236980
                      10           2.546842
                      11           2.714035
                      12           2.487526
                      13           2.815397
                      14           2.579146
                      15           1.150049
24         25         16           0.358654
23         26         17           0.722216
                      18           1.001097
24         25         19           0.312880
                      20          -0.004982
                      21          -0.531237
